# Module 2: Memory and Learning Systems 🧠
*Building Retail AI Agents That Remember and Improve*

**Duration**: 50 minutes  
**Level**: Intermediate  
**Domain**: Retail Operations (Walmart)

## 🎯 Learning Objectives

By the end of this module, you'll understand:

1. **Memory Architecture**: Four types of memory systems for retail AI
2. **Working Memory**: Real-time inventory and customer interactions
3. **Episodic Memory**: Learning from past sales events and seasons
4. **Semantic Memory**: Product knowledge and store operations
5. **Procedural Memory**: Optimized workflows and best practices
6. **LLM Integration**: Using Ollama for intelligent memory retrieval
7. **Adaptive Learning**: Improving from Black Friday to everyday ops

## 🏪 Retail Context

Imagine you're building an AI assistant for Walmart that:
- Remembers customer shopping patterns
- Learns from seasonal trends
- Optimizes inventory management
- Improves store operations
- Adapts to local market conditions

## 📚 Understanding Memory Types in Retail

### 1. **Working Memory** (Current Operations)
- Active customer in store needing help
- Current inventory levels and alerts
- Today's promotions and pricing
- Real-time store traffic

### 2. **Episodic Memory** (Past Events)
- Last year's Black Friday performance
- Successful product launches
- Customer complaint resolutions
- Seasonal shopping patterns

### 3. **Semantic Memory** (Retail Knowledge)
- Product specifications and relationships
- Store layout and department info
- Supplier details and lead times
- Walmart policies and procedures

### 4. **Procedural Memory** (How-To Skills)
- Inventory reordering workflows
- Customer service protocols
- Price matching procedures
- Seasonal transition strategies

## 🔧 Environment Setup

In [ ]:
# Configuration for Ollama LLM (matching Module 1)
MODEL_NAME = "qwen2.5:7b-instruct-q4_K_M"
OLLAMA_BASE_URL = "http://localhost:11434"

# Core imports
import json
import requests
import numpy as np
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, field
from collections import deque, defaultdict
from enum import Enum
import time
import math
import random
import re

# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Test Ollama connection
try:
    response = requests.get(f"{OLLAMA_BASE_URL}/api/tags", timeout=5)
    if response.status_code == 200:
        print("✅ Ollama server is running")
        models = response.json().get('models', [])
        model_names = [model['name'] for model in models]
        if MODEL_NAME in model_names:
            print(f"✅ {MODEL_NAME} is available for intelligent memory operations")
            OLLAMA_AVAILABLE = True
        else:
            print(f"❌ {MODEL_NAME} not found. Run: ollama pull qwen2.5:7b-instruct-q4_K_M")
            OLLAMA_AVAILABLE = False
    else:
        print(f"❌ Ollama server responded with status {response.status_code}")
        OLLAMA_AVAILABLE = False
except:
    print("⚠️ Cannot connect to Ollama. Memory system will use basic operations.")
    print("To enable intelligent memory, run: ollama serve")
    OLLAMA_AVAILABLE = False

print("\n🏪 Walmart Retail AI Memory System Ready!")
print("📍 Location: Bentonville, AR Supercenter #100")

## 🏗️ Part 1: Memory Architecture

Let's build a comprehensive memory system from the ground up.

In [ ]:
# Define memory types for retail context
class MemoryType(Enum):
    """Types of memory in our retail AI system"""
    WORKING = "working"      # Current store state
    EPISODIC = "episodic"    # Past events and outcomes
    SEMANTIC = "semantic"    # Product and policy knowledge
    PROCEDURAL = "procedural" # Operational workflows

@dataclass
class RetailMemoryItem:
    """Base class for all retail memory items"""
    content: Any                    # The memory content
    timestamp: datetime             # When it occurred
    importance: float               # Business impact (0-1)
    department: str = "General"     # Which department
    store_id: str = "#100"         # Which store
    access_count: int = 0          # How often referenced
    last_accessed: Optional[datetime] = None
    decay_rate: float = 0.1        # How fast it becomes less relevant
    memory_type: MemoryType = MemoryType.EPISODIC
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def access(self):
        """Update access statistics"""
        self.access_count += 1
        self.last_accessed = datetime.now()
        # Accessing reinforces memory importance
        self.importance = min(1.0, self.importance * 1.1)
    
    def get_strength(self) -> float:
        """Calculate memory strength using Ebbinghaus forgetting curve"""
        if not self.last_accessed:
            self.last_accessed = self.timestamp
            
        time_since_access = (datetime.now() - self.last_accessed).total_seconds()
        hours_passed = time_since_access / 3600
        
        # Ebbinghaus forgetting curve: R = e^(-t/S)
        # Where S is stability (affected by importance and access count)
        stability = (1 + self.importance) * (1 + math.log1p(self.access_count))
        retention = math.exp(-hours_passed / (stability * 24))  # 24 hours base stability
        
        # Apply importance weighting
        strength = retention * self.importance
        
        # Boost for recent access (recency effect)
        if hours_passed < 1:
            strength *= 1.2
        elif hours_passed < 24:
            strength *= 1.1
            
        # Boost for frequently accessed items (frequency effect)
        if self.access_count > 5:
            strength *= 1.1
        elif self.access_count > 10:
            strength *= 1.2
            
        return min(1.0, strength)

@dataclass
class RetailEpisode(RetailMemoryItem):
    """Memory of a specific retail event"""
    event_type: str = ""           # sale, return, complaint, etc.
    products_involved: List[str] = field(default_factory=list)
    customer_segment: str = ""     # regular, premium, new
    outcome: str = ""              # What happened
    revenue_impact: float = 0.0    # Dollar impact
    success: bool = True           # Was it successful?
    
    def __post_init__(self):
        self.memory_type = MemoryType.EPISODIC

@dataclass
class ProductKnowledge(RetailMemoryItem):
    """Semantic memory about products and categories"""
    product_id: str = ""           # SKU or product ID
    category: str = ""             # Product category
    relations: Dict[str, List[str]] = field(default_factory=dict)  # Related products
    attributes: Dict[str, Any] = field(default_factory=dict)  # Price, features, etc.
    supplier_info: Dict[str, Any] = field(default_factory=dict)
    confidence: float = 1.0        # How accurate this info is
    
    def __post_init__(self):
        self.memory_type = MemoryType.SEMANTIC

@dataclass 
class RetailProcedure(RetailMemoryItem):
    """Memory of retail operational procedures"""
    procedure_name: str = ""       # What procedure
    trigger_conditions: List[str] = field(default_factory=list)  # When to use
    steps: List[str] = field(default_factory=list)  # How to execute
    success_rate: float = 0.0      # Historical effectiveness
    execution_count: int = 0       # Times used
    average_time: float = 0.0      # Minutes to complete
    cost_savings: float = 0.0      # Average $ saved/earned
    
    def __post_init__(self):
        self.memory_type = MemoryType.PROCEDURAL
        
    def update_performance(self, success: bool, time_taken: float, savings: float = 0):
        """Update procedure performance metrics"""
        self.execution_count += 1
        # Running average for success rate
        self.success_rate = ((self.success_rate * (self.execution_count - 1)) + 
                           (1.0 if success else 0.0)) / self.execution_count
        # Running average for time
        self.average_time = ((self.average_time * (self.execution_count - 1)) + 
                           time_taken) / self.execution_count
        # Track cost savings
        self.cost_savings = ((self.cost_savings * (self.execution_count - 1)) + 
                           savings) / self.execution_count

@dataclass
class CustomerInteraction(RetailMemoryItem):
    """Special episodic memory for customer interactions"""
    interaction_type: str = ""     # inquiry, complaint, praise
    customer_mood: str = ""        # happy, frustrated, neutral
    resolution: str = ""           # How it was resolved
    satisfaction_score: float = 0.0  # 1-5 rating
    associate_id: str = ""         # Who helped
    
    def __post_init__(self):
        self.memory_type = MemoryType.EPISODIC
        # Customer interactions are always important
        self.importance = max(0.7, self.importance)

print("✅ Retail memory structures defined!")
print(f"Memory types: {[t.value for t in MemoryType]}")
print("\n🏪 Specialized for Walmart operations:")
print("  - Product relationships and inventory")
print("  - Customer interaction tracking")
print("  - Procedural efficiency metrics")
print("  - Revenue impact analysis")

## 🧠 Part 2: LLM Integration for Intelligent Memory

Using Ollama to enhance memory retrieval and consolidation.

In [ ]:
class OllamaLLM:
    """LLM interface for intelligent memory operations"""
    
    def __init__(self, model: str = MODEL_NAME, temperature: float = 0.7):
        self.model = model
        self.temperature = temperature
        self.base_url = OLLAMA_BASE_URL
        self.available = OLLAMA_AVAILABLE
        
    def generate(self, prompt: str, system: str = "") -> str:
        """Generate response from LLM"""
        if not self.available:
            return "LLM not available"
            
        full_prompt = f"{system}\n\nUser: {prompt}\n\nAssistant:" if system else prompt
        
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json={
                    "model": self.model,
                    "prompt": full_prompt,
                    "temperature": self.temperature,
                    "stream": False
                },
                timeout=30
            )
            
            if response.status_code == 200:
                return response.json().get('response', '')
            else:
                return f"Error: {response.status_code}"
                
        except Exception as e:
            return f"Error: {str(e)}"
    
    def analyze_memory_relevance(self, query: str, memory_content: str) -> float:
        """Use LLM to score memory relevance (0-1)"""
        if not self.available:
            return 0.5  # Default relevance
            
        prompt = f"""Rate the relevance of this memory to the query on a scale of 0-10.
        Query: {query}
        Memory: {memory_content}
        
        Only respond with a number 0-10."""
        
        response = self.generate(prompt)
        try:
            # Extract number from response
            numbers = re.findall(r'\d+', response)
            if numbers:
                score = min(10, max(0, int(numbers[0]))) / 10.0
                return score
        except:
            pass
        return 0.5  # Default relevance
    
    def extract_query_intent(self, query: str) -> Dict[str, Any]:
        """Extract intent from query"""
        if not self.available:
            return {"department": "General", "type": "general", "urgency": "medium"}
            
        prompt = f"""Analyze this retail query and extract key intent:
        Query: {query}
        
        Identify:
        1. Department (Electronics, Grocery, etc.)
        2. Type (product, procedure, customer service, etc.)
        3. Urgency (high, medium, low)
        
        Be concise."""
        
        response = self.generate(prompt)
        
        # Simple parsing
        intent = {
            "department": "General",
            "type": "general",
            "urgency": "medium"
        }
        
        # Extract department
        for dept in ["Electronics", "Grocery", "Apparel", "Pharmacy", "Operations"]:
            if dept.lower() in response.lower():
                intent["department"] = dept
                break
                
        return intent

print("✅ LLM interface ready for intelligent memory operations")
print(f"🤖 LLM Available: {OLLAMA_AVAILABLE}")

## 💾 Part 3: Walmart Memory Manager

A sophisticated memory system with LLM-enhanced retrieval and consolidation.

In [ ]:
class WalmartMemoryManager:
    """Manages retail memories with business intelligence and LLM integration"""
    
    def __init__(self, 
                 store_id: str = "#100",
                 working_memory_size: int = 10,
                 consolidation_threshold: float = 0.7,
                 use_llm: bool = True):
        self.store_id = store_id
        self.consolidation_threshold = consolidation_threshold
        self.use_llm = use_llm and OLLAMA_AVAILABLE
        
        # Initialize LLM if available
        if self.use_llm:
            self.llm = OllamaLLM()
            print("🤖 LLM-enhanced memory operations enabled")
        else:
            print("📝 Using basic memory operations (no LLM)")
        
        # Working memory - limited size queue
        self.working_memory = deque(maxlen=working_memory_size)
        
        # Long-term memories
        self.episodes: List[RetailEpisode] = []
        self.product_knowledge: Dict[str, ProductKnowledge] = {}
        self.procedures: Dict[str, RetailProcedure] = {}
        self.customer_interactions: List[CustomerInteraction] = []
        
        # Indices for fast retrieval
        self.product_index = defaultdict(list)  # category -> products
        self.seasonal_index = defaultdict(list)  # season -> episodes
        self.department_index = defaultdict(list)  # dept -> memories
        
        # Business metrics
        self.total_revenue_impact = 0.0
        self.customer_satisfaction_avg = 4.0
        self.total_memories = 0
        
        # Initialize with some Walmart knowledge
        self._initialize_walmart_knowledge()
    
    def add_to_working_memory(self, item: Any):
        """Add item to working memory with intelligent consolidation"""
        self.working_memory.append(item)
        
        # Automatic consolidation based on importance threshold
        if isinstance(item, RetailMemoryItem):
            # Check consolidation threshold
            if item.importance >= self.consolidation_threshold:
                print(f"  💾 Auto-consolidating to long-term memory (importance: {item.importance:.2f})")
                self._consolidate_memory(item)
            
            # Also consolidate if working memory is getting full
            elif len(self.working_memory) >= self.working_memory.maxlen - 2:
                # Find most important items to consolidate
                memory_items = [m for m in self.working_memory if isinstance(m, RetailMemoryItem)]
                if memory_items:
                    memory_items.sort(key=lambda m: m.importance, reverse=True)
                    for mem in memory_items[:3]:  # Consolidate top 3
                        if mem.importance >= self.consolidation_threshold * 0.8:
                            self._consolidate_memory(mem)
    
    def _consolidate_memory(self, memory: RetailMemoryItem):
        """Move important memories from working to long-term storage"""
        if isinstance(memory, RetailEpisode):
            self.episodes.append(memory)
            # Index by context words
            for word in memory.outcome.lower().split():
                self.seasonal_index[word].append(memory)
                
        elif isinstance(memory, ProductKnowledge):
            # Merge with existing knowledge if exists
            if memory.product_id in self.product_knowledge:
                existing = self.product_knowledge[memory.product_id]
                # Merge relations
                for rel_type, concepts in memory.relations.items():
                    if rel_type in existing.relations:
                        existing.relations[rel_type].extend(concepts)
                    else:
                        existing.relations[rel_type] = concepts
                # Update confidence (weighted average)
                total_weight = existing.importance + memory.importance
                existing.confidence = ((existing.confidence * existing.importance + 
                                      memory.confidence * memory.importance) / total_weight)
                existing.importance = min(1.0, existing.importance + 0.1)
            else:
                self.product_knowledge[memory.product_id] = memory
                self.product_index[memory.category].append(memory.product_id)
                        
        elif isinstance(memory, RetailProcedure):
            # Merge with existing procedure if exists
            if memory.procedure_name in self.procedures:
                existing = self.procedures[memory.procedure_name]
                # Keep the more successful procedure
                if memory.success_rate > existing.success_rate:
                    self.procedures[memory.procedure_name] = memory
            else:
                self.procedures[memory.procedure_name] = memory
                
        elif isinstance(memory, CustomerInteraction):
            self.customer_interactions.append(memory)
        
        self.total_memories += 1
    
    def retrieve_relevant_memories(self, query: str, context: Dict[str, Any] = None,
                                 limit: int = 5) -> List[RetailMemoryItem]:
        """Retrieve memories with intelligent relevance scoring"""
        print(f"\n🔍 Retrieving memories for: '{query}'")
        
        # Extract query intent using LLM if available
        query_intent = {}
        if self.use_llm:
            query_intent = self.llm.extract_query_intent(query)
            print(f"  📊 Intent: {query_intent}")
        
        query_lower = query.lower()
        
        # Score all memories
        scored_memories = []
        
        # Score episodes
        for episode in self.episodes:
            score = self._score_memory_relevance(episode, query, query_intent)
            if score > 0.1:  # Threshold
                scored_memories.append((episode, score))
        
        # Score product knowledge
        for product_id, knowledge in self.product_knowledge.items():
            score = self._score_memory_relevance(knowledge, query, query_intent)
            if score > 0.1:
                scored_memories.append((knowledge, score))
        
        # Score procedures
        for proc_name, procedure in self.procedures.items():
            score = self._score_memory_relevance(procedure, query, query_intent)
            if score > 0.1:
                scored_memories.append((procedure, score))
        
        # Score recent customer interactions
        for interaction in self.customer_interactions[-10:]:
            score = self._score_memory_relevance(interaction, query, query_intent)
            if score > 0.1:
                scored_memories.append((interaction, score))
        
        # Sort by combined score (relevance * strength)
        scored_memories.sort(key=lambda x: x[1] * x[0].get_strength(), reverse=True)
        
        # Get top memories and update access
        top_memories = []
        for memory, score in scored_memories[:limit]:
            memory.access()
            top_memories.append(memory)
            print(f"  ✓ Retrieved: {memory.content[:50]}... (relevance: {score:.2f})")
        
        return top_memories
    
    def _score_memory_relevance(self, memory: RetailMemoryItem, query: str, 
                               intent: Dict[str, Any]) -> float:
        """Score memory relevance using multiple factors"""
        score = 0.0
        query_words = set(query.lower().split())
        
        # 1. Keyword matching (basic)
        if isinstance(memory, RetailEpisode):
            # Check products
            for product in memory.products_involved:
                if any(word in product.lower() for word in query_words):
                    score += 0.3
            # Check outcome
            if any(word in memory.outcome.lower() for word in query_words):
                score += 0.2
                
        elif isinstance(memory, ProductKnowledge):
            if any(word in memory.product_id.lower() for word in query_words):
                score += 0.4
            if any(word in memory.category.lower() for word in query_words):
                score += 0.3
                
        elif isinstance(memory, RetailProcedure):
            if any(word in memory.procedure_name.lower() for word in query_words):
                score += 0.5
            # Check trigger conditions
            for trigger in memory.trigger_conditions:
                if any(word in trigger.lower() for word in query_words):
                    score += 0.2
                    
        # 2. Department matching
        if intent.get("department") == memory.department:
            score += 0.2
            
        # 3. LLM-based semantic similarity (if available)
        if self.use_llm and score > 0:
            llm_score = self.llm.analyze_memory_relevance(query, memory.content)
            score = (score + llm_score) / 2  # Average with LLM score
            
        # 4. Recency boost for customer interactions
        if isinstance(memory, CustomerInteraction):
            hours_old = (datetime.now() - memory.timestamp).total_seconds() / 3600
            if hours_old < 24:
                score *= 1.2
                
        # 5. Success/failure relevance
        if "problem" in query.lower() or "issue" in query.lower():
            if hasattr(memory, "success") and not memory.success:
                score *= 1.3  # Boost failed memories for problem queries
                
        return min(1.0, score)
    
    def _initialize_walmart_knowledge(self):
        """Pre-load essential Walmart operational knowledge"""
        # Price match procedure
        price_match = RetailProcedure(
            content="Walmart Price Match Guarantee procedure",
            timestamp=datetime.now(),
            importance=0.9,
            department="Customer Service",
            procedure_name="price_match",
            trigger_conditions=["customer shows lower price", "competitor ad"],
            steps=[
                "Verify competitor price on current ad",
                "Check if item is identical (brand, size, model)",
                "Confirm competitor has item in stock",
                "Apply price match at register",
                "Log price match in system"
            ],
            success_rate=0.95,
            execution_count=1000,
            average_time=3.5,
            cost_savings=-5.50  # Average loss per match, but retains customer
        )
        self.procedures["price_match"] = price_match
        
        # Black Friday prep procedure
        black_friday = RetailProcedure(
            content="Black Friday preparation workflow",
            timestamp=datetime.now(),
            importance=1.0,
            department="Operations",
            procedure_name="black_friday_prep",
            trigger_conditions=["2 weeks before Black Friday"],
            steps=[
                "Review last year's sales data and pain points",
                "Double-check inventory levels for doorbusters",
                "Schedule extra staff for all departments",
                "Set up queue management systems",
                "Prepare online pickup area for increased volume",
                "Test all POS systems and backups",
                "Brief staff on crowd control procedures"
            ],
            success_rate=0.92,
            execution_count=5,
            average_time=480,  # 8 hours
            cost_savings=125000  # Smooth operations save/earn this much
        )
        self.procedures["black_friday_prep"] = black_friday
        
        print("📚 Loaded Walmart operational procedures")
    
    # [Continue with all other methods from the original: remember_sale, remember_customer_interaction, etc.]
    
    def remember_sale(self, products: List[str], amount: float, 
                     customer_type: str = "regular", special_event: str = ""):
        """Store memory of a sale transaction"""
        episode = RetailEpisode(
            content=f"Sale of {len(products)} items for ${amount:.2f}",
            timestamp=datetime.now(),
            importance=min(1.0, amount / 1000),  # Higher sales more important
            department=self._get_department(products[0] if products else "General"),
            event_type="sale",
            products_involved=products,
            customer_segment=customer_type,
            outcome=f"Completed sale ${amount:.2f}",
            revenue_impact=amount,
            success=True
        )
        
        if special_event:
            episode.metadata["event"] = special_event
            self.seasonal_index[special_event].append(episode)
            
        self.add_to_working_memory(episode)
        self.total_revenue_impact += amount
        
        # Update product relationships
        if len(products) > 1:
            self._update_product_relationships(products)
            
        return episode
    
    def remember_customer_interaction(self, interaction_type: str, 
                                    department: str, resolution: str,
                                    satisfaction: float, mood: str = "neutral"):
        """Store customer service interaction"""
        interaction = CustomerInteraction(
            content=f"{interaction_type} in {department}: {resolution}",
            timestamp=datetime.now(),
            importance=0.8 if interaction_type == "complaint" else 0.6,
            department=department,
            interaction_type=interaction_type,
            customer_mood=mood,
            resolution=resolution,
            satisfaction_score=satisfaction,
            associate_id=f"A{random.randint(1000, 9999)}"
        )
        
        self.add_to_working_memory(interaction)
        
        # Update satisfaction average
        total_interactions = len(self.customer_interactions)
        self.customer_satisfaction_avg = (
            (self.customer_satisfaction_avg * total_interactions + satisfaction) 
            / (total_interactions + 1)
        )
        
        return interaction
    
    def learn_product_fact(self, product_id: str, category: str, 
                          attributes: Dict, relations: Dict[str, List[str]] = None):
        """Store product knowledge"""
        knowledge = ProductKnowledge(
            content=f"Product {product_id} in {category}",
            timestamp=datetime.now(),
            importance=0.7,
            department=self._get_department(category),
            product_id=product_id,
            category=category,
            attributes=attributes,
            relations=relations or {},
            confidence=0.95
        )
        
        self.add_to_working_memory(knowledge)
        return knowledge
    
    def _get_department(self, item: str) -> str:
        """Map items to Walmart departments"""
        dept_map = {
            "grocery": "Grocery",
            "electronics": "Electronics",
            "apparel": "Apparel",
            "home": "Home & Garden",
            "pharmacy": "Pharmacy",
            "auto": "Auto Care",
            "toys": "Toys",
            "sports": "Sports & Outdoors"
        }
        
        item_lower = item.lower()
        for key, dept in dept_map.items():
            if key in item_lower:
                return dept
        return "General Merchandise"
    
    def _update_product_relationships(self, products: List[str]):
        """Learn which products are frequently bought together"""
        for i, product1 in enumerate(products):
            if product1 in self.product_knowledge:
                knowledge = self.product_knowledge[product1]
                if "frequently_bought_with" not in knowledge.relations:
                    knowledge.relations["frequently_bought_with"] = []
                    
                for product2 in products[i+1:]:
                    if product2 not in knowledge.relations["frequently_bought_with"]:
                        knowledge.relations["frequently_bought_with"].append(product2)
    
    def get_department_insights(self, department: str) -> Dict[str, Any]:
        """Get insights for a specific department"""
        dept_episodes = [e for e in self.episodes if e.department == department]
        
        if not dept_episodes:
            return {"message": f"No data for {department}"}
            
        total_revenue = sum(e.revenue_impact for e in dept_episodes)
        avg_transaction = total_revenue / len(dept_episodes) if dept_episodes else 0
        
        # Find best selling products
        product_sales = defaultdict(int)
        for episode in dept_episodes:
            for product in episode.products_involved:
                product_sales[product] += 1
                
        top_products = sorted(product_sales.items(), 
                            key=lambda x: x[1], reverse=True)[:5]
        
        return {
            "department": department,
            "total_revenue": total_revenue,
            "transaction_count": len(dept_episodes),
            "avg_transaction_size": avg_transaction,
            "top_products": top_products,
            "time_range": {
                "earliest": min(e.timestamp for e in dept_episodes),
                "latest": max(e.timestamp for e in dept_episodes)
            }
        }
    
    def visualize_retail_intelligence(self):
        """Create retail-specific memory visualizations"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle(f'Walmart Store {self.store_id} Memory Analytics', fontsize=16)
        
        # Department revenue distribution
        dept_revenue = defaultdict(float)
        for episode in self.episodes:
            if episode.revenue_impact > 0:
                dept_revenue[episode.department] += episode.revenue_impact
                
        if dept_revenue:
            depts, revenues = zip(*dept_revenue.items())
            ax1.pie(revenues, labels=depts, autopct='%1.1f%%', startangle=90)
            ax1.set_title("Revenue by Department")
        
        # Customer satisfaction trend
        if self.customer_interactions:
            interactions = self.customer_interactions[-20:]  # Last 20
            satisfaction_scores = [i.satisfaction_score for i in interactions]
            ax2.plot(satisfaction_scores, marker='o', linewidth=2, markersize=8)
            ax2.axhline(y=4.0, color='g', linestyle='--', label='Target (4.0)')
            ax2.set_xlabel("Recent Interactions")
            ax2.set_ylabel("Satisfaction Score")
            ax2.set_title("Customer Satisfaction Trend")
            ax2.set_ylim(1, 5)
            ax2.legend()
        
        # Procedure effectiveness
        if self.procedures:
            procs = list(self.procedures.values())
            proc_names = [p.procedure_name for p in procs]
            success_rates = [p.success_rate for p in procs]
            savings = [p.cost_savings for p in procs]
            
            x = np.arange(len(proc_names))
            width = 0.35
            
            ax3.bar(x - width/2, success_rates, width, label='Success Rate')
            ax3.set_xlabel('Procedures')
            ax3.set_ylabel('Success Rate', color='b')
            ax3.set_title('Procedure Performance')
            ax3.set_xticks(x)
            ax3.set_xticklabels(proc_names, rotation=45, ha='right')
            ax3.set_ylim(0, 1.1)
            
            # Add savings on secondary axis
            ax3_twin = ax3.twinx()
            ax3_twin.bar(x + width/2, savings, width, label='Avg Savings', color='g')
            ax3_twin.set_ylabel('Average Savings ($)', color='g')
        
        # Time-based activity
        if self.episodes:
            # Group by hour of day
            hour_activity = defaultdict(int)
            for episode in self.episodes:
                hour = episode.timestamp.hour
                hour_activity[hour] += 1
                
            hours = list(range(24))
            activity = [hour_activity.get(h, 0) for h in hours]
            
            ax4.bar(hours, activity, color='skyblue')
            ax4.set_xlabel('Hour of Day')
            ax4.set_ylabel('Transaction Count')
            ax4.set_title('Store Activity by Hour')
            ax4.set_xticks(range(0, 24, 2))
        
        plt.tight_layout()
        plt.show()

# Create Walmart memory manager
walmart_memory = WalmartMemoryManager(store_id="#100")
print(f"\n✅ Walmart Memory Manager initialized for Store {walmart_memory.store_id}")
print(f"📊 Customer satisfaction baseline: {walmart_memory.customer_satisfaction_avg:.1f}/5.0")
print(f"📋 Pre-loaded procedures: {list(walmart_memory.procedures.keys())}")

## 🧪 Part 4: Retail Scenarios in Action

Let's simulate real Walmart scenarios to see our memory system work.

In [ ]:
# Scenario 1: Black Friday Shopping Pattern
print("🛒 Simulating Black Friday 2023...\n")

# Early morning doorbusters
black_friday_sales = [
    (["Samsung 65in TV", "Soundbar", "HDMI Cable"], 899.99, "deal_hunter"),
    (["PS5 Console", "Extra Controller", "Game Bundle"], 549.99, "gamer"),
    (["Instant Pot", "Air Fryer", "Kitchen Scale"], 129.99, "regular"),
    (["Apple iPad", "Apple Pencil", "Case"], 449.99, "premium"),
    (["Kids Bike", "Helmet", "Training Wheels"], 89.99, "family"),
]

for products, amount, customer_type in black_friday_sales:
    episode = walmart_memory.remember_sale(
        products=products,
        amount=amount,
        customer_type=customer_type,
        special_event="Black Friday 2023"
    )
    print(f"✅ {episode.content}")

print(f"\n💰 Total Black Friday revenue so far: ${walmart_memory.total_revenue_impact:,.2f}")

In [ ]:
# Scenario 2: Customer Service Interactions
print("\n👥 Customer Service Desk Activity...\n")

interactions = [
    ("complaint", "Electronics", "Exchanged defective headphones", 3.5, "frustrated"),
    ("inquiry", "Grocery", "Located gluten-free products in aisle 7", 5.0, "happy"),
    ("return", "Apparel", "Processed return without receipt per policy", 4.0, "neutral"),
    ("complaint", "Customer Service", "Applied price match to competitor ad", 4.5, "satisfied"),
    ("praise", "Electronics", "Customer thanked associate for TV setup help", 5.0, "delighted"),
]

for int_type, dept, resolution, satisfaction, mood in interactions:
    interaction = walmart_memory.remember_customer_interaction(
        interaction_type=int_type,
        department=dept,
        resolution=resolution,
        satisfaction=satisfaction,
        mood=mood
    )
    emoji = "😊" if satisfaction >= 4 else "😐" if satisfaction >= 3 else "😞"
    print(f"{emoji} [{int_type}] {dept}: {resolution} (Score: {satisfaction}/5)")

print(f"\n📈 Updated satisfaction average: {walmart_memory.customer_satisfaction_avg:.2f}/5.0")

In [ ]:
# Scenario 3: Learning Product Relationships
print("\n📦 Loading Product Knowledge...\n")

# Popular products with relationships
products = [
    ("TV-SAMSUNG-65-Q80A", "Electronics", 
     {"price": 899.99, "brand": "Samsung", "size": "65 inch", "display": "QLED"},
     {"accessories": ["Wall Mount", "HDMI Cable"], "competes_with": ["LG OLED65"]}),
    
    ("FOOD-CEREAL-CHEERIOS", "Grocery",
     {"price": 3.98, "brand": "General Mills", "size": "18oz", "healthy": True},
     {"pairs_well": ["Milk", "Bananas"], "alternatives": ["Lucky Charms", "Corn Flakes"]}),
    
    ("TOY-LEGO-STARWARS", "Toys",
     {"price": 79.99, "brand": "LEGO", "pieces": 500, "age_range": "8-14"},
     {"theme_related": ["Star Wars Action Figures", "SW Lightsaber"], "gift_with": ["Gift Wrap"]}),
]

for product_id, category, attributes, relations in products:
    knowledge = walmart_memory.learn_product_fact(
        product_id=product_id,
        category=category,
        attributes=attributes,
        relations=relations
    )
    print(f"📚 Learned: {product_id} - ${attributes['price']} ({category})")
    print(f"   Relations: {', '.join(relations.keys())}")

In [ ]:
# Test memory retrieval with LLM
print("\n🔍 Testing Intelligent Memory Retrieval...\n")

queries = [
    "How did Black Friday go?",
    "Customer complaints in Electronics",
    "Samsung TV related products",
    "Price match procedure",
]

for query in queries:
    memories = walmart_memory.retrieve_relevant_memories(query, limit=3)
    
    if memories:
        print(f"\n📌 Found {len(memories)} relevant memories")
        for i, memory in enumerate(memories, 1):
            if isinstance(memory, RetailEpisode):
                print(f"{i}. [Sale] {memory.content} - Revenue: ${memory.revenue_impact:.2f}")
            elif isinstance(memory, CustomerInteraction):
                print(f"{i}. [Service] {memory.interaction_type}: {memory.resolution}")
            elif isinstance(memory, ProductKnowledge):
                print(f"{i}. [Product] {memory.product_id} - {memory.category}")
            elif isinstance(memory, RetailProcedure):
                print(f"{i}. [Procedure] {memory.procedure_name} - Success: {memory.success_rate:.0%}")
    else:
        print("   No relevant memories found.")

In [ ]:
# Visualize retail intelligence
print("\n📊 Generating Retail Intelligence Dashboard...\n")
walmart_memory.visualize_retail_intelligence()

## 🤖 Part 5: Walmart Assistant with Learning

An AI assistant that learns from store operations and improves over time.

In [ ]:
class WalmartAssistant:
    """AI Assistant for Walmart operations with learning capabilities"""
    
    def __init__(self, name: str, store_id: str):
        self.name = name
        self.store_id = store_id
        self.memory = WalmartMemoryManager(store_id=store_id)
        self.shift_start = datetime.now()
        
    def handle_customer_query(self, query: str, department: str = "General"):
        """Process customer queries using memory"""
        print(f"\n🤖 {self.name}: Processing customer query...")
        print(f"📍 Department: {department}")
        print(f"❓ Query: '{query}'")
        
        # Retrieve relevant memories
        memories = self.memory.retrieve_relevant_memories(query)
        
        response = self._generate_response(query, memories, department)
        
        # Simulate customer satisfaction
        satisfaction = 4.0 if memories else 3.0
        if "price match" in query.lower() and "price_match" in self.memory.procedures:
            satisfaction = 4.5  # Following procedure improves satisfaction
            
        # Remember this interaction
        self.memory.remember_customer_interaction(
            interaction_type="inquiry",
            department=department,
            resolution=response,
            satisfaction=satisfaction
        )
        
        return response
    
    def _generate_response(self, query: str, memories: List, department: str) -> str:
        """Generate response based on memories"""
        query_lower = query.lower()
        
        # Check for price match
        if "price match" in query_lower or "lower price" in query_lower:
            if "price_match" in self.memory.procedures:
                proc = self.memory.procedures["price_match"]
                return (f"I can help with price matching! Here's our process: "
                       f"{', '.join(proc.steps[:3])}. "
                       f"This typically takes {proc.average_time:.0f} minutes.")
        
        # Check for product queries
        for memory in memories:
            if isinstance(memory, ProductKnowledge):
                attrs = memory.attributes
                return (f"I found {memory.product_id}: "
                       f"${attrs.get('price', 'N/A')} - {attrs.get('brand', 'Generic')} "
                       f"in {memory.category}. Located in {department}.")
        
        # Check for Black Friday info
        if "black friday" in query_lower:
            bf_memories = self.memory.seasonal_index.get("Black Friday 2023", [])
            if bf_memories:
                total_sales = sum(m.revenue_impact for m in bf_memories)
                return (f"Black Friday was successful! We processed {len(bf_memories)} "
                       f"major transactions. Our Electronics doorbusters were very popular!")
        
        # Default response
        return f"Let me find someone in {department} to help you with that specific question."
    
    def analyze_shift_performance(self):
        """Analyze performance during shift"""
        shift_duration = (datetime.now() - self.shift_start).total_seconds() / 3600
        
        print(f"\n📊 {self.name}'s Shift Analysis")
        print(f"⏰ Shift duration: {shift_duration:.1f} hours\n")
        
        # Get department insights
        for dept in ["Electronics", "Grocery", "Customer Service"]:
            insights = self.memory.get_department_insights(dept)
            if "total_revenue" in insights:
                print(f"📍 {dept}:")
                print(f"   Revenue: ${insights['total_revenue']:,.2f}")
                print(f"   Transactions: {insights['transaction_count']}")
                if insights['top_products']:
                    top_product = insights['top_products'][0]
                    print(f"   Top item: {top_product[0]}")
                print()
        
        # Customer satisfaction
        print(f"😊 Customer Satisfaction: {self.memory.customer_satisfaction_avg:.2f}/5.0")
        
        # Most used procedures
        if self.memory.procedures:
            print(f"\n🔧 Available Procedures:")
            for name, proc in self.memory.procedures.items():
                print(f"   - {name}: {proc.success_rate:.0%} success rate")

# Create Walmart AI assistant
assistant = WalmartAssistant("Sam", "#100")
print(f"✅ Walmart Assistant '{assistant.name}' is ready!")
print(f"🏪 Assigned to Store {assistant.store_id}")

## 🚀 Part 6: Assistant in Action

Let's see our Walmart assistant handle real scenarios.

In [ ]:
# Simulate a busy retail day
print("🌅 Starting retail simulation...\n")

# Handle customer queries throughout the day
customer_queries = [
    ("9:15 AM", "Where can I find Samsung TVs?", "Electronics"),
    ("10:30 AM", "Do you price match Amazon?", "Customer Service"),
    ("11:45 AM", "Looking for gluten-free cereal", "Grocery"),
    ("2:00 PM", "When is your next Black Friday?", "General"),
    ("3:30 PM", "My Samsung TV from Black Friday isn't working", "Electronics"),
]

for time_str, query, dept in customer_queries:
    print(f"\n=== {time_str} ===")
    response = assistant.handle_customer_query(query, dept)
    print(f"💬 Response: {response}")
    time.sleep(0.5)  # Simulate time passing

## 🎯 Hands-On Exercise: Advanced Memory Types

Create specialized memory types for seasonal patterns and competitive intelligence.

In [ ]:
# Exercise: Create a Seasonal Pattern Memory with Advanced Analytics
@dataclass
class SeasonalPattern(RetailMemoryItem):
    """
    Memory type that learns seasonal patterns for better inventory planning.
    
    This exercise teaches:
    - Pattern recognition from historical data
    - Weather correlation analysis
    - Promotion effectiveness tracking
    - Multi-year trend analysis
    """
    season: str = ""               # spring, summer, fall, winter
    year: int = 2024              # Which year
    top_categories: List[str] = field(default_factory=list)
    weather_correlation: Dict[str, float] = field(default_factory=dict)
    promotion_effectiveness: Dict[str, float] = field(default_factory=dict)
    total_revenue: float = 0.0
    yoy_growth: float = 0.0       # Year-over-year growth
    peak_shopping_days: List[str] = field(default_factory=list)
    inventory_recommendations: Dict[str, float] = field(default_factory=dict)
    
    def __post_init__(self):
        self.memory_type = MemoryType.SEMANTIC  # Patterns are knowledge
        # Seasonal patterns are very important for planning
        self.importance = 0.9
        self.decay_rate = 0.02  # Decay very slowly - yearly relevance

# Advanced implementation with pattern analysis
def analyze_seasonal_trends(memory_manager: WalmartMemoryManager, 
                          season: str, year: int) -> SeasonalPattern:
    """Analyze and store seasonal patterns with business insights"""
    # Define season months
    season_months = {
        "winter": [12, 1, 2],
        "spring": [3, 4, 5],
        "summer": [6, 7, 8],
        "fall": [9, 10, 11]
    }
    
    # Analyze episodes from that season
    seasonal_episodes = []
    category_revenue = defaultdict(float)
    daily_revenue = defaultdict(float)
    promo_performance = defaultdict(list)
    
    for episode in memory_manager.episodes:
        if episode.timestamp.month in season_months.get(season, []):
            seasonal_episodes.append(episode)
            category_revenue[episode.department] += episode.revenue_impact
            
            # Track daily patterns
            day_key = episode.timestamp.strftime("%A")  # Day of week
            daily_revenue[day_key] += episode.revenue_impact
            
            # Track promotion effectiveness
            if "event" in episode.metadata:
                promo_type = episode.metadata["event"]
                promo_performance[promo_type].append(episode.revenue_impact)
    
    # Calculate top categories
    top_cats = sorted(category_revenue.items(), key=lambda x: x[1], reverse=True)
    
    # Find peak shopping days
    peak_days = sorted(daily_revenue.items(), key=lambda x: x[1], reverse=True)[:3]
    
    # Calculate promotion effectiveness
    promo_effectiveness = {}
    for promo, revenues in promo_performance.items():
        if revenues:
            avg_revenue = sum(revenues) / len(revenues)
            # Compare to non-promo average
            base_avg = sum(e.revenue_impact for e in seasonal_episodes 
                          if "event" not in e.metadata) / max(1, len(seasonal_episodes))
            effectiveness = (avg_revenue / base_avg - 1) * 100 if base_avg > 0 else 0
            promo_effectiveness[promo] = effectiveness
    
    # Weather correlations (simulated - in reality would use weather data)
    weather_correlation = {
        "temperature": 0.7 if season == "summer" else -0.3,
        "precipitation": -0.5 if season == "summer" else 0.2,
        "snow": 0.8 if season == "winter" else -0.9
    }
    
    # Inventory recommendations
    inventory_recs = {}
    for cat, revenue in top_cats[:5]:
        # Recommend 20% increase for top performers
        inventory_recs[cat] = 1.2 if revenue > 10000 else 1.0
    
    # Create seasonal pattern
    pattern = SeasonalPattern(
        content=f"Seasonal pattern for {season} {year}",
        timestamp=datetime.now(),
        importance=0.9,
        season=season,
        year=year,
        top_categories=[cat for cat, _ in top_cats[:5]],
        total_revenue=sum(category_revenue.values()),
        weather_correlation=weather_correlation,
        promotion_effectiveness=promo_effectiveness,
        peak_shopping_days=[day for day, _ in peak_days],
        inventory_recommendations=inventory_recs,
        yoy_growth=random.uniform(-5, 15)  # Simulated
    )
    
    return pattern

# TODO: Student Challenge - Extend this to:
# 1. Compare patterns across multiple years
# 2. Predict next season's trends
# 3. Suggest optimal promotion timing
# 4. Identify weather-sensitive products

# Test the enhanced implementation
print("🍂 Analyzing Fall 2023 patterns with advanced metrics...")
fall_pattern = analyze_seasonal_trends(assistant.memory, "fall", 2023)
print(f"\nTop categories: {', '.join(fall_pattern.top_categories[:3])}")
print(f"Total revenue: ${fall_pattern.total_revenue:,.2f}")
print(f"Peak shopping days: {', '.join(fall_pattern.peak_shopping_days)}")

if fall_pattern.promotion_effectiveness:
    best_promo = max(fall_pattern.promotion_effectiveness.items(), key=lambda x: x[1])
    print(f"Most effective promotion: {best_promo[0]} (+{best_promo[1]:.1f}% revenue)")

print("\n📈 Inventory Recommendations:")
for cat, multiplier in fall_pattern.inventory_recommendations.items():
    if multiplier > 1.0:
        print(f"  - {cat}: Increase by {(multiplier-1)*100:.0f}%")

## 📊 Summary and Key Takeaways

### What You've Learned:

1. **Retail-Specific Memory Architecture**:
   - Working Memory: Current store state and active customers
   - Episodic Memory: Sales transactions and customer interactions
   - Semantic Memory: Product knowledge and relationships
   - Procedural Memory: Operational workflows with ROI tracking

2. **LLM Integration**:
   - Ollama/Qwen2.5 for intelligent memory retrieval
   - Semantic similarity scoring beyond keyword matching
   - Query intent extraction for better relevance
   - Graceful fallback when LLM unavailable

3. **Advanced Memory Features**:
   - Ebbinghaus forgetting curve implementation
   - Automatic consolidation based on importance
   - Multi-factor relevance scoring
   - Recency and frequency effects

4. **Business Intelligence**:
   - Revenue impact tracking per memory
   - Customer satisfaction monitoring
   - Department-level insights
   - Seasonal pattern recognition

5. **Learning from Retail Operations**:
   - Product relationship discovery (frequently bought together)
   - Procedure effectiveness measurement
   - Customer service improvement tracking
   - Black Friday and seasonal preparation

### 🚀 Real-World Applications:

This memory system could power:
- **Smart Inventory Management**: Predictive restocking
- **Customer Service Bots**: Context-aware assistance
- **Revenue Optimization**: Data-driven pricing and promotions
- **Operational Excellence**: Learning from successful procedures

### 💡 Challenge Exercise:

Implement a **Competitive Intelligence Memory** that:
1. Tracks competitor pricing and promotions
2. Learns which price matches drive sales
3. Identifies market trends and opportunities
4. Suggests competitive strategies

### 📈 Business Impact:

With proper implementation, this system could:
- Increase customer satisfaction by 15%
- Reduce inventory waste by 20%
- Improve checkout efficiency by 25%
- Boost seasonal revenue by 10%

Ready for Module 3? We'll connect these agents to real databases and APIs! 🚀